In [1]:
import pandas as pd
import warnings, pickle, os, json
warnings.filterwarnings("ignore")

gap = 40
min_change = 20

In [2]:
input_ = {
    "work_id": "FC236606T1-2",
    "op": 1,
    "model_id": "20230807120000000",
    "speed": "1000",
    "l_angle_ori": "39",
    "l_weight_ori": "535",
    "f_angle_ori": "209",
    "f_weight_ori": "297"
}

In [3]:
work_id = input_["work_id"]
op = input_["op"]
model_id = input_["model_id"]
speed = int(input_["speed"])
l_angle_ori  = int(input_["l_angle_ori"])
l_weight_ori = round(float(input_["l_weight_ori"]))
f_angle_ori  = int(input_["f_angle_ori"])
f_weight_ori = round(float(input_["f_weight_ori"]))
work_id, op, speed, l_angle_ori, l_weight_ori, f_angle_ori, f_weight_ori

('FC236606T1-2', 1, 1000, 39, 535, 209, 297)

In [4]:
root = r"C:\Users\tzuli\Documents\python\teco"

# 取得predict位置
pred_path = os.path.join(root, "data", "predict")        
os.makedirs(pred_path, exist_ok = True)
output_json = os.path.join(pred_path, "output.json")

# 取得model位置     
model_detail = os.path.join(root, "data", "train", model_id, "model")

### Load model

In [5]:
side = "L"
features = pickle.load(open(os.path.join(model_detail, f"feat_order.pkl"), "rb"))
l_outlier_boundary = pickle.load(open(os.path.join(model_detail, f"{side}_outlier_boundary.pkl"), "rb"))
l_skew_feat = pickle.load(open(os.path.join(model_detail, f"{side}_skew_feat.pkl"), "rb"))
l_pt = pickle.load(open(os.path.join(model_detail, f"{side}_power_tf.pkl"), "rb"))
l_scaler = pickle.load(open(os.path.join(model_detail, f"{side}_scaler.pkl"), "rb"))
l_model = pickle.load(open(os.path.join(model_detail, f"{side}_model.pkl"), "rb"))

side = "F"
f_outlier_boundary = pickle.load(open(os.path.join(model_detail, f"{side}_outlier_boundary.pkl"), "rb"))
f_skew_feat = pickle.load(open(os.path.join(model_detail, f"{side}_skew_feat.pkl"), "rb"))
f_pt = pickle.load(open(os.path.join(model_detail, f"{side}_power_tf.pkl"), "rb"))
f_scaler = pickle.load(open(os.path.join(model_detail, f"{side}_scaler.pkl"), "rb"))
f_model = pickle.load(open(os.path.join(model_detail, f"{side}_model.pkl"), "rb"))

### Generate candidate values

In [6]:
l_weight_change, f_weight_change = [], []
for i in range(gap+1):
    l_weight_change.extend([max(min_change, l_weight_ori + i), max(min_change, l_weight_ori - i)])
    f_weight_change.extend([max(min_change, f_weight_ori + i), max(min_change, f_weight_ori - i)])
l_weight_change = list(set(l_weight_change))
f_weight_change = list(set(f_weight_change))

print(f"l_weight_change = {l_weight_change}")
print(f"f_weight_change = {f_weight_change}")

l_weight_change = [512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511]
f_weight_change = [257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337]


In [7]:
l_angle_change0, f_angle_change0 = [], []
for i in range(0, gap+1, 10):
    l_angle_change0.extend([l_angle_ori + i, l_angle_ori - i])
    f_angle_change0.extend([f_angle_ori + i, f_angle_ori - i])
l_angle_change0 = list(set(l_angle_change0))
f_angle_change0 = list(set(f_angle_change0))


l_angle_change = []
for angle in l_angle_change0:
    if angle > 360:
        l_angle_change.append(angle - 360)
    elif angle < 0:
        l_angle_change.append(360 + angle)
    else:
        l_angle_change.append(angle)


f_angle_change = []
for angle in f_angle_change0:
    if angle > 360:
        f_angle_change.append(angle - 360)
    elif angle < 0:
        f_angle_change.append(360 + angle)
    else:
        f_angle_change.append(angle)
        
print(f"l_angle_change = {l_angle_change}")
print(f"f_angle_change = {f_angle_change}")

l_angle_change = [69, 39, 9, 79, 49, 19, 59, 29, 359]
f_angle_change = [229, 199, 169, 239, 209, 179, 249, 219, 189]


### Generate candidate combinations

In [8]:
# 循環生成所有组合
all_combinations = []
for a in l_angle_change :
    for b in l_weight_change:
        for c in f_angle_change:
            for d in f_weight_change:
                x = [speed, l_angle_ori, l_weight_ori, f_angle_ori, f_weight_ori, a, b, c, d]
                all_combinations.append(x)
                
len(all_combinations)

531441

In [9]:
import pandas as pd
a = pd.array([1,2,3])
a

<IntegerArray>
[1, 2, 3]
Length: 3, dtype: Int64

In [10]:
df_comb = pd.DataFrame(
    all_combinations, 
    columns = ['平衡轉速', '初始_L側角度', '初始_L側不平衡量', '初始_F側角度', '初始_F側不平衡量', '平衡_L側角度', '平衡_L側配重', '平衡_F側角度', '平衡_F側配重']
    )
df_pred = df_comb.copy()
df_pred.head(3)

,平衡轉速,初始_L側角度,初始_L側不平衡量,初始_F側角度,初始_F側不平衡量,平衡_L側角度,平衡_L側配重,平衡_F側角度,平衡_F側配重
0,1000,39,535,209,297,69,512,229,257
1,1000,39,535,209,297,69,512,229,258
2,1000,39,535,209,297,69,512,229,259


### Generate features

In [11]:
df_pred["預期_F側不平衡量"] = df_pred["初始_F側不平衡量"] - df_pred["平衡_F側配重"]
df_pred["預期_L側不平衡量"] = df_pred["初始_L側不平衡量"] - df_pred["平衡_L側配重"]

df_pred["初始_不平衡量_diff"] = abs(df_pred["初始_L側不平衡量"] - df_pred["初始_F側不平衡量"])
df_pred["平衡_不平衡量_diff"] = abs(df_pred["平衡_L側配重"] - df_pred["平衡_F側配重"])

diff = abs(df_pred["初始_L側角度"] - df_pred["初始_F側角度"])
df_pred["初始_角度_diff"] = diff.apply(lambda X: min(X, 360 - X))
diff = abs(df_pred["平衡_L側角度"] - df_pred["平衡_F側角度"])
df_pred["平衡_角度_diff"] = diff.apply(lambda X: min(X, 360 - X))
df_pred.head(3)

,平衡轉速,初始_L側角度,初始_L側不平衡量,初始_F側角度,初始_F側不平衡量,平衡_L側角度,平衡_L側配重,平衡_F側角度,平衡_F側配重,預期_F側不平衡量,預期_L側不平衡量,初始_不平衡量_diff,平衡_不平衡量_diff,初始_角度_diff,平衡_角度_diff
0,1000,39,535,209,297,69,512,229,257,40,23,238,255,170,160
1,1000,39,535,209,297,69,512,229,258,39,23,238,254,170,160
2,1000,39,535,209,297,69,512,229,259,38,23,238,253,170,160


In [12]:
l_df, f_df = df_pred.copy(), df_pred.copy()

### Feature engineering

In [13]:
for col in features[1:]:    
    max_ = l_outlier_boundary[col]["max"]
    max_index = l_df.index[l_df[col] > max_].to_list()
    l_df.loc[max_index, col] = max_

    min_ = l_outlier_boundary[col]["min"]
    min_index = l_df.index[l_df[col] < min_].to_list()
    l_df.loc[min_index, col] = min_

    max_ = f_outlier_boundary[col]["max"]
    max_index = f_df.index[f_df[col] > max_].to_list()
    f_df.loc[max_index, col] = max_

    min_ = f_outlier_boundary[col]["min"]
    min_index = f_df.index[f_df[col] < min_].to_list()
    f_df.loc[min_index, col] = min_

In [14]:
l_df[l_skew_feat] = l_pt.transform(l_df[l_skew_feat])
f_df[f_skew_feat] = f_pt.transform(f_df[f_skew_feat])
l_df[features]    = l_scaler.transform(l_df[features])
f_df[features]    = f_scaler.transform(f_df[features])

### Predict

In [15]:
pred_l = l_model.predict(l_df)
pred_f = f_model.predict(f_df)

In [16]:
df_comb["最終_L側不平衡量"] = pred_l
df_comb["最終_F側不平衡量"] = pred_f

### Find best combination

In [17]:
df_comb["總不平衡量"] = df_comb.eval("(abs(最終_L側不平衡量) + abs(最終_F側不平衡量))")
# 先嚴格篩選，在寬鬆篩選
df_comb1 = df_comb.query("((平衡_L側配重 > 0) and (平衡_F側配重 > 0)) and ((最終_L側不平衡量 >= 0.1) and (最終_F側不平衡量 >= 0.1))")
if len(df_comb1) == 0:
    df_comb1 = df_comb.query("(最終_L側不平衡量 >= 0.1) and (最終_F側不平衡量 > 0.1)")
if len(df_comb1) == 0:
    df_comb1 = df_comb.copy()
comb_index = df_comb1["總不平衡量"].idxmin()
result = df_comb.loc[[comb_index]]
result

,平衡轉速,初始_L側角度,初始_L側不平衡量,初始_F側角度,初始_F側不平衡量,平衡_L側角度,平衡_L側配重,平衡_F側角度,平衡_F側配重,最終_L側不平衡量,最終_F側不平衡量,總不平衡量
198482,1000,39,535,209,297,79,541,169,289,4.892395,10.401699,15.294094


In [18]:
result = result[['平衡_L側角度', '平衡_L側配重', '平衡_F側角度', '平衡_F側配重']]
result = result.astype(str)
result.columns = ["l_angle_pred", "l_weight_pred", "f_angle_pred", "f_weight_pred"]
result = result.to_dict(orient = "records")[0]

### Save output

In [19]:
result = {
    "status":   "success",
    "reason":   "",
    "work_id":  work_id,
    "op":       op,
    "model_id": model_id,
    "predict":  result
}
result

{'status': 'success',
 'reason': '',
 'work_id': 'FC236606T1-2',
 'op': 1,
 'model_id': '20230807120000000',
 'predict': {'l_angle_pred': '79',
  'l_weight_pred': '541',
  'f_angle_pred': '169',
  'f_weight_pred': '289'}}

In [20]:
with open(output_json, 'w') as file:
    json.dump(result, file, indent = 4)